In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import multiprocessing as mp
plt.style.use("fivethirtyeight")
#libraries used for Association rule mining
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
train=pd.read_csv("subset_train_data.csv")

In [72]:
test=pd.read_csv("test_ver2.csv",low_memory=False,na_values=[""," ","?"])

In [5]:
#pd.set_option('display.max_columns', 500)
#test

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,NaN,1.0,A,S,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,NaN,1.0,A,S,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO
5,2016-06-28,1170550,N,ES,V,22,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KHE,N,1,8.0,BARCELONA,0,NA,03 - UNIVERSITARIO
6,2016-06-28,1170552,N,ES,H,51,2013-08-28,0,34,1,NaN,1.0,A,S,N,NaN,KFC,N,1,35.0,"PALMAS, LAS",1,96395.88,02 - PARTICULARES
7,2016-06-28,1170553,N,ES,H,22,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KHE,N,1,45.0,TOLEDO,0,NA,03 - UNIVERSITARIO
8,2016-06-28,1170555,N,ES,V,22,2013-08-28,0,34,1,NaN,1.0,I,S,N,NaN,KHE,N,1,28.0,MADRID,0,NA,03 - UNIVERSITARIO
9,2016-06-28,1170557,N,ES,H,22,2013-08-28,0,34,1,NaN,1.0,A,S,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,68322.72,03 - UNIVERSITARIO


In [4]:
(test.isna().sum()/len(test))*100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.000000
pais_residencia           0.000000
sexo                      0.000538
age                       0.000000
fecha_alta                0.000000
ind_nuevo                 0.000000
antiguedad                0.000000
indrel                    0.000000
ult_fec_cli_1t           99.818957
indrel_1mes               0.002474
tiprel_1mes               0.002474
indresi                   0.000000
indext                    0.000000
conyuemp                 99.988813
canal_entrada             0.223856
indfall                   0.000000
tipodom                   0.000000
cod_prov                  0.429855
nomprov                   0.429855
ind_actividad_cliente     0.000000
renta                     0.000000
segmento                  0.241821
dtype: float64

In [73]:
test=test.drop(['ult_fec_cli_1t', 'conyuemp'], axis = 1)

In [74]:
test=test[test['pais_residencia']=='ES']

In [75]:
test=test[test['ind_actividad_cliente']==1]

In [76]:
test['segmento']=test["segmento"].str.split("-", n = 1, expand = True)[1]

In [77]:
len(train['ncodpers'].unique())

KeyError: 'ncodpers'

In [10]:
train.rename(columns={'ncodpers':'Customer_code','ind_empleado':'Employee index','pais_residencia':'CustomerCountryResidence','sexo':'sex','age':'Age','fecha_alta':'Date_of_first_account_holder','ind_nuevo':'New_customer_Index','antiguedad':'Customer_seniority_(in months)','indrel':'Primary_customer_status','indrel_1mes':'Customer_type_at_beginning_of_month','tiprel_1mes':'Customer relation type at the beginning of the month','indresi':'Residence index','indext':'Foreigner_index','canal_entrada':'channel_used_by_customer','indfall':'Deceased index','tipodom':'Addres type','cod_prov':'Province code','nomprov':'Province name','ind_actividad_cliente':'Activity index','renta':'Gross income','segmento':'segmentation','ind_ahor_fin_ult1':'Saving Account','ind_aval_fin_ult1':'Guarantees','ind_cco_fin_ult1':'Current Accounts','ind_cder_fin_ult1':'Derivada Account','ind_cno_fin_ult1':'Payroll Account','ind_ctju_fin_ult1':'Junior Account','ind_ctma_fin_ult1':'Más particular Account','ind_ctop_fin_ult1':'particular Account','ind_ctpp_fin_ult1':'particular Plus Account','ind_deco_fin_ult1':'Short-term deposits','ind_deme_fin_ult1':'Medium-term deposits','ind_dela_fin_ult1':'Long-term deposits','ind_ecue_fin_ult1':'e-account','ind_fond_fin_ult1':'Funds','ind_hip_fin_ult1':'Mortgage','ind_plan_fin_ult1':'Pensions','ind_pres_fin_ult1':'Loans','ind_reca_fin_ult1':'Taxes','ind_tjcr_fin_ult1':'Credit Card','ind_valo_fin_ult1':'Securities','ind_viv_fin_ult1':'Home Account','ind_nomina_ult1':'Payroll','ind_nom_pens_ult1':'Pensions','ind_recibo_ult1':'Direct Debit'}, inplace=True)

In [11]:
train.columns

Index(['Unnamed: 0', 'fecha_dato', 'Customer_code', 'Employee index',
       'CustomerCountryResidence', 'sex', 'Age',
       'Date_of_first_account_holder', 'New_customer_Index',
       'Customer_seniority_(in months)', 'Primary_customer_status',
       'Customer_type_at_beginning_of_month',
       'Customer relation type at the beginning of the month',
       'Residence index', 'Foreigner_index', 'channel_used_by_customer',
       'Deceased index', 'Addres type', 'Province code', 'Province name',
       'Activity index', 'Gross income', 'segmentation', 'Saving Account',
       'Guarantees', 'Current Accounts', 'Derivada Account', 'Payroll Account',
       'Junior Account', 'Más particular Account', 'particular Account',
       'particular Plus Account', 'Short-term deposits',
       'Medium-term deposits', 'Long-term deposits', 'e-account', 'Funds',
       'Mortgage', 'Pensions', 'Loans', 'Taxes', 'Credit Card', 'Securities',
       'Home Account', 'Payroll', 'Pensions', 'Direct Debi

In [12]:
class popularity_recommendation_model():
    
    #class variables
    MODEL_NAME = 'Popularity'
    data=0
    #constructor
    def __init__(self, data):
        self.data = data
    
    #Class functions
    
    #Function to get model name.    
    def get_model_name(self):
        return self.MODEL_NAME
    
    #Function to get most popular product.
    def get_popular_products(self):
        pop=self.data[['Customer_code','Saving Account','Guarantees', 'Current Accounts', 'Derivada Account', 'Payroll Account','Junior Account', 'Más particular Account', 'particular Account','particular Plus Account', 'Short-term deposits','Medium-term deposits', 'Long-term deposits', 'e-account', 'Funds','Mortgage', 'Pensions', 'Loans', 'Taxes', 'Credit Card', 'Securities','Home Account', 'Payroll', 'Pensions', 'Direct Debit']]
        pop['Customer_code']=pop['Customer_code'].astype('str')
        pop=pop.set_index('Customer_code')
        #converting float data to (int)
        pop['Payroll']=pop['Payroll'].astype('int')
        pop['Pensions']=pop['Pensions'].astype('int')
        frequent_itemsets = apriori(pop, min_support=0.07, use_colnames=True)
        return frequent_itemsets.sort_values(['support'], ascending=[False]).reset_index()['itemsets']
    
    def get_association_between_the_product(self):
        pop=self.data[['Customer_code','Saving Account','Guarantees', 'Current Accounts', 'Derivada Account', 'Payroll Account','Junior Account', 'Más particular Account', 'particular Account','particular Plus Account', 'Short-term deposits','Medium-term deposits', 'Long-term deposits', 'e-account', 'Funds','Mortgage', 'Pensions', 'Loans', 'Taxes', 'Credit Card', 'Securities','Home Account', 'Payroll', 'Pensions', 'Direct Debit']]
        pop['Customer_code']=pop['Customer_code'].astype('str')
        pop=pop.set_index('Customer_code')
        #converting float data to (int)
        pop['Payroll']=pop['Payroll'].astype('int')
        pop['Pensions']=pop['Pensions'].astype('int')
        frequent_itemsets = apriori(pop, min_support=0.08, use_colnames=True)
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
        return association_rules(frequent_itemsets, metric="lift", min_threshold=1)
        
    
    #Function for recommendation
    #def Recommend(self,sample):

In [15]:
ob=popularity_recommendation_model(train)
ob.get_popular_products()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0                                    (Current Accounts)
1                                        (Direct Debit)
2                                  (particular Account)
3                                     (Payroll Account)
4                                           (e-account)
5                      (Direct Debit, Current Accounts)
6                (Current Accounts, particular Account)
7                       (Direct Debit, Payroll Account)
8                                            (Pensions)
9                                            (Pensions)
10                                           (Pensions)
11                          (Pensions, Payroll Account)
12                          (Pensions, Payroll Account)
13                          (Pensions, Payroll Account)
14                                            (Payroll)
15                                  (Payroll, Pensions)
16                                  (Payroll, Pensions)
17                                  (Payroll, Pe

In [16]:
ob.get_association_between_the_product()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Current Accounts),(particular Account),0.732716,0.200913,0.154021,0.210205,1.046252,0.006809,1.011766
1,(particular Account),(Current Accounts),0.200913,0.732716,0.154021,0.766606,1.046252,0.006809,1.145204
2,(Pensions),(Payroll Account),0.138272,0.185429,0.129995,0.940141,5.070086,0.104356,13.608181
3,(Payroll Account),(Pensions),0.185429,0.138272,0.129995,0.701052,5.070086,0.104356,2.882533
4,(Payroll),(Payroll Account),0.127275,0.185429,0.119910,0.942130,5.080812,0.096309,14.075840
5,(Payroll Account),(Payroll),0.185429,0.127275,0.119910,0.646662,5.080812,0.096309,2.469945
6,(Direct Debit),(Payroll Account),0.292117,0.185429,0.138870,0.475393,2.563749,0.084703,1.552727
7,(Payroll Account),(Direct Debit),0.185429,0.292117,0.138870,0.748914,2.563749,0.084703,2.819289
8,(e-account),(Direct Debit),0.178375,0.292117,0.089300,0.500631,1.713805,0.037194,1.417556
9,(Direct Debit),(e-account),0.292117,0.178375,0.089300,0.305700,1.713805,0.037194,1.183386


In [78]:
test.rename(columns={'ncodpers':'Customer_code','ind_empleado':'Employee index','pais_residencia':'CustomerCountryResidence','sexo':'sex','age':'Age','fecha_alta':'Date_of_first_account_holder','ind_nuevo':'New_customer_Index','antiguedad':'Customer_seniority_(in months)','indrel':'Primary_customer_status','indrel_1mes':'Customer_type_at_beginning_of_month','tiprel_1mes':'Customer relation type at the beginning of the month','indresi':'Residence index','indext':'Foreigner_index','canal_entrada':'channel_used_by_customer','indfall':'Deceased index','tipodom':'Addres type','cod_prov':'Province code','nomprov':'Province name','ind_actividad_cliente':'Activity index','renta':'Gross income','segmento':'segmentation'}, inplace=True)

In [18]:
pd.set_option('display.max_columns', 500)
train.head(3)

,Unnamed: 0,fecha_dato,Customer_code,Employee index,CustomerCountryResidence,sex,Age,Date_of_first_account_holder,New_customer_Index,Customer_seniority_(in months),Primary_customer_status,Customer_type_at_beginning_of_month,Customer relation type at the beginning of the month,Residence index,Foreigner_index,channel_used_by_customer,Deceased index,Addres type,Province code,Province name,Activity index,Gross income,segmentation,Saving Account,Guarantees,Current Accounts,Derivada Account,Payroll Account,Junior Account,Más particular Account,particular Account,particular Plus Account,Short-term deposits,Medium-term deposits,Long-term deposits,e-account,Funds,Mortgage,Pensions,Loans,Taxes,Credit Card,Securities,Home Account,Payroll,Pensions,Direct Debit
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,1.0,A,S,N,KHL,N,1.0,29.0,MALAGA,1,87218.10,PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,7,2015-01-28,1050617,N,ES,H,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1,113316.66,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,12,2015-01-28,1050623,N,ES,H,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,49.0,ZAMORA,1,113538.81,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [20]:
#train.columns
len(train['Customer_code'].unique())

330193

In [21]:
len(test['Customer_code'].unique())

393167

In [28]:
from scipy import spatial

a = [1, 2]
b = [2, 4]
c = [2.5, 4]
d = [4.5, 5]

print(spatial.distance.euclidean(c, a))
print(spatial.distance.euclidean(c, b))
print(spatial.distance.euclidean(c, d))
print(spatial.distance.euclidean(a, b))

2.5
0.5
2.23606797749979
2.23606797749979


In [26]:
from scipy import spatial
a = [1, 2]
b = [2, 4]
c = [2.5, 4]
d = [4.5, 5]

print(spatial.distance.cosine(c,a))
print(spatial.distance.cosine(c,b))
print(spatial.distance.cosine(c,d))
print(spatial.distance.cosine(a,b))

0.004504527406047898
0.004504527406047898
0.015137225946083022
0.0


In [50]:
# load_data.py

import pandas as pd
from surprise import Dataset
from surprise import Reader

# This is the same data that was plotted for similarity earlier
# with one new user "E" who has rated only movie 1
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

ratings_dict
df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))
#df
# Loads Pandas dataframe
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
#data
# Loads the builtin Movielens-100k data
movielens = Dataset.load_builtin('ml-100k')

In [51]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [53]:
#rom load_data import data
#rom recommender import algo
trainingSet= data.build_full_trainset()
algo.fit(trainingSet)
rediction = algo.predict('E', 2)
rediction.est

Computing the cosine similarity matrix...
Done computing similarity matrix.


4.15

In [56]:
train['fecha_dato'] = pd.to_datetime(train['fecha_dato'])
train['Date_of_first_account_holder'] = pd.to_datetime(train['Date_of_first_account_holder'])

In [99]:
#diff
print(len(test['Customer_code'].unique())-len(train['Customer_code'].unique()))
#print(len(train['Customer_code'].unique()))

62974


In [112]:
print("train")
print(len(train['Customer_code'].unique()))
print('test')
print(len(test['Customer_code'].unique()))
print("difference",62974)
print("removing all new users data :- ")
len(test['Customer_code'].unique())-62974

train
330193
test
393167
difference 62974
removing all new users data :- 


330193

In [119]:
#len(train[train['Customer_code'].isin(test['Customer_code'].unique())].Customer_code.unique())
len(test[test['Customer_code'].isin(train['Customer_code'].unique())].Customer_code.unique())

301782

In [120]:
test.dtypes

fecha_dato                                               object
Customer_code                                             int64
Employee index                                           object
CustomerCountryResidence                                 object
sex                                                      object
Age                                                       int64
Date_of_first_account_holder                             object
New_customer_Index                                        int64
Customer_seniority_(in months)                            int64
Primary_customer_status                                   int64
Customer_type_at_beginning_of_month                     float64
Customer relation type at the beginning of the month     object
Residence index                                          object
Foreigner_index                                          object
channel_used_by_customer                                 object
Deceased index                          

In [121]:
train

,Unnamed: 0,fecha_dato,Customer_code,Employee index,CustomerCountryResidence,sex,Age,Date_of_first_account_holder,New_customer_Index,Customer_seniority_(in months),Primary_customer_status,Customer_type_at_beginning_of_month,Customer relation type at the beginning of the month,Residence index,Foreigner_index,channel_used_by_customer,Deceased index,Addres type,Province code,Province name,Activity index,Gross income,segmentation,Saving Account,Guarantees,Current Accounts,Derivada Account,Payroll Account,Junior Account,Más particular Account,particular Account,particular Plus Account,Short-term deposits,Medium-term deposits,Long-term deposits,e-account,Funds,Mortgage,Pensions,Loans,Taxes,Credit Card,Securities,Home Account,Payroll,Pensions,Direct Debit
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,1.0,A,S,N,KHL,N,1.0,29.0,MALAGA,1,87218.10,PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,7,2015-01-28,1050617,N,ES,H,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1,113316.66,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,12,2015-01-28,1050623,N,ES,H,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,49.0,ZAMORA,1,113538.81,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,13,2015-01-28,1050624,N,ES,H,65,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1,61605.09,PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,16,2015-01-28,1050610,N,ES,V,24,2012-08-10,0.0,35,1.0,1.0,I,S,N,KHE,N,1.0,37.0,SALAMANCA,1,68318.46,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
5,18,2015-01-28,1050609,N,ES,H,22,2012-08-10,0.0,35,1.0,1.0,I,S,N,KFA,N,1.0,13.0,CIUDAD REAL,1,73432.47,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
6,20,2015-01-28,1050582,N,ES,V,28,2012-08-10,0.0,35,1.0,1.0,I,S,N,KHE,N,1.0,13.0,CIUDAD REAL,1,64620.57,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
7,21,2015-01-28,1050586,N,ES,V,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,13.0,CIUDAD REAL,1,64194.99,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1
8,24,2015-01-28,1050591,N,ES,H,23,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,13.0,CIUDAD REAL,1,58728.39,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9,26,2015-01-28,1050595,N,ES,V,25,2012-08-10,0.0,35,1.0,1.0,A,S,N,KHE,N,1.0,5.0,AVILA,1,86863.38,UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
